## Example Usage of AbstractRatingSet and related Classes ##

### Setup ###

In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import json
import platform
import time
from datetime import datetime
from typing import Optional, cast

from hec import CwmsDataStore, DeleteAction, DssDataStore, HecTime, Location, TimeSeries
from hec.rating import AbstractRatingSet, LocalRatingSet, ReferenceRatingSet

with open(os.path.join(os.path.dirname(os.getcwd()), ".vscode", "launch.json")) as f:
    config = json.load(f)
for key, value in config["configurations"][0].get("env", {}).items():
    if key == "REQUESTS_CA_BUNDLE" or key.startswith("cda_api_"):
        os.environ[key] = value

### Catalog CWMS Rating Information ###

In [2]:
with CwmsDataStore.open(time_zone="US/Pacific") as db:

    # ---------------------------------------- #
    # catalog rating templates in the database #
    # ---------------------------------------- #
    pattern = "*Opening*"
    print(f"\nCataloging rating templates matching {pattern}")
    catalog = db.catalog(
        "RATING_TEMPLATE",
        pattern=pattern,
        fields="name,lookup-methods",
        header=True,
    )
    for item in catalog:
        print(f"\t{item}")
    # --------------------------------------------- #
    # catalog rating specifications in the database #
    # --------------------------------------------- #
    pattern = "keys*"
    print(f"\nCataloging rating specifications matching {pattern}")
    catalog = db.catalog(
        "RATING_SPECIFICATION",
        pattern=pattern,
        fields="name,lookup-methods,rounding-specs,description,effective-date",
        header=True,
    )
    first = True
    headers: list[str] = 20 * [""]
    for item in catalog:
        parts = item.split("\t")
        if first:
            headers = parts[:]
            first = False
            continue
        for i in range(len(parts)):
            print(f"\t{headers[i]} = {parts[i]}")
        print("")
    # ------------------------------- #
    # catalog ratings in the database #
    # ------------------------------- #
    pattern = "keys*"
    print(f"\nCataloging ratings matching {pattern}")
    catalog = db.catalog(
        "RATING",
        pattern=pattern,
        fields="name, create-date, effective-date, units, type",
        header=True,
    )
    for item in catalog:
        print(f"\t{item}")


Cataloging rating templates matching *Opening*
	#name	lookup-methods
	%-Opening,Elev;Flow.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	%-Opening-Auxiliary_Low_Flow_Gates,Elev;Flow-Auxiliary_Low_Flow_Gates.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	%-Opening-Conduit_Gates,Elev;Flow-Conduit_Gates.Standard	LINEAR,NULL,NULL;LINEAR,NULL,NULL
	%-Opening-Low_Flow_Gates,Elev;Flow-Low_Flow_Gates.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	Count-Conduit_Gates,Opening-Conduit_Gates,Elev;Flow-Conduit_Gates.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	Count-Conduit_Gates,Opening-Conduit_Gates,Elev;Flow.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	Count-Low_Flow_Gates,%-Opening,Elev;Flow-Low_Flow_Gates.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,NEAREST
	Count-Low_Flow_Gates,%-Opening-Low_Flow_Gates,Elev;Flow-Low_Flow_Gates.Standard	LINEAR,NEAREST,NEAREST;LINEAR,NEAREST,

### Use AbstractRatingSet (ReferenceRatingSet or LocalRatingSet) from CWMS to Rate and ReverseRate TimeSeries

In [3]:
with CwmsDataStore.open() as db:
    # ----------------------------------- #
    # get a time series from the database #
    # ----------------------------------- #
    db.time_window = "t-1d, t"
    elev_ts = cast(TimeSeries, db.retrieve("KEYS.Elev.Inst.1Hour.0.Ccp-Rev"))
    print(elev_ts)
    print(elev_ts.data)
    times: dict[str, dict[str, str]] = {
        "retrieve": {},
        "rate": {},
        "reverse_rate": {},
        "total": {},
    }
    methods = ["REFERENCE", "LAZY", "EAGER"]
    for method in methods:
        # ------------------------------------------------ #
        # get a rating set from the database               #
        #                                                  #
        # method="REFERENCE" is valid only for CWMS data   #
        # stores and returns a ReferenceRatingSet that     #
        #   1. sends data to the database to be rated      #
        #   2. retrieves the rated values                  #
        #                                                  #
        # methods "LAZY" (default) and "EAGER" return a    #
        # LocalRatingSet that perform the ratings in local #
        # code. the difference is:                         #
        # - "LAZY" doesn't retrieve table data until it is #
        #   actually needed (prevents loading lots of      #
        #   data that is never used)                       #
        # - "EAGER"  retrieves all of the table data up    #
        #   front                                          #
        # ------------------------------------------------ #
        ts1 = datetime.now()
        rating_set = cast(
            AbstractRatingSet,
            db.retrieve("KEYS.Elev;Stor.Linear.Production", method=method),
        )
        ts2 = datetime.now()
        times["retrieve"][method] = str(ts2 - ts1)
        # -------------------- #
        # rate the time series #
        # -------------------- #
        ts3 = datetime.now()
        stor_ts = cast(TimeSeries, rating_set.rate(elev_ts))
        ts4 = datetime.now()
        times["rate"][method] = str(ts4 - ts3)
        print(f"\n{stor_ts}")
        print(stor_ts.data)
        # ---------------------------------- #
        # reverse rate the rated time series #
        # ---------------------------------- #
        ts5 = datetime.now()
        elev_ts2 = cast(TimeSeries, rating_set.reverse_rate(stor_ts))
        ts6 = datetime.now()
        times["reverse_rate"][method] = str(ts6 - ts5)
        times["total"][method] = str((ts2 - ts1) + (ts4 - ts3) + (ts6 - ts5))
        print(f"\n{elev_ts2}")
        print(elev_ts2.data)

    print("\n\t\tREFERENCE\tLAZY\t\tEAGER")
    print(f"retrieve\t{chr(9).join([times['retrieve'][m] for m in methods])}")
    print(f"rate\t\t{chr(9).join([times['rate'][m] for m in methods])}")
    print(f"reverse_rate\t{chr(9).join([times['reverse_rate'][m] for m in methods])}")
    print(f"\n\t\t{chr(9).join([times['total'][m] for m in methods])}")

KEYS.Elev.Inst.1Hour.0.Ccp-Rev 24 values in ft
                            value  quality
time                                      
2025-10-21 12:00:00-05:00  725.11        0
2025-10-21 13:00:00-05:00  725.08        0
2025-10-21 14:00:00-05:00  725.09        0
2025-10-21 15:00:00-05:00  725.05        0
2025-10-21 16:00:00-05:00  725.01        0
2025-10-21 17:00:00-05:00  724.96        0
2025-10-21 18:00:00-05:00  724.94        0
2025-10-21 19:00:00-05:00  724.89        0
2025-10-21 20:00:00-05:00  724.86        0
2025-10-21 21:00:00-05:00  724.86        0
2025-10-21 22:00:00-05:00  724.87        0
2025-10-21 23:00:00-05:00  724.87        0
2025-10-22 00:00:00-05:00  724.90        0
2025-10-22 01:00:00-05:00  724.90        0
2025-10-22 02:00:00-05:00  724.92        0
2025-10-22 03:00:00-05:00  724.92        0
2025-10-22 04:00:00-05:00  724.94        0
2025-10-22 05:00:00-05:00  724.95        0
2025-10-22 06:00:00-05:00  724.96        0
2025-10-22 07:00:00-05:00  724.97        0
2025-10

### Copy LocalRatingSet from CWMS to HEC-DSS and catalog in DSS

In [4]:
with CwmsDataStore.open() as db:
    rating_set = cast(
        LocalRatingSet, db.retrieve("KEYS.Elev;Stor.Linear.Production", method="EAGER")
    )

dss_file_name = (
    "C:/TEMP/rating_set.dss"
    if platform.system() == "Windows"
    else "/var/tmp/rating.dss"
)
with DssDataStore.open(dss_file_name, read_only=False) as dss:
    dss.store(rating_set)
    print(f"Rating IDs in DSS file {dss.name}")
    for rating_id in dss.catalog("RATING"):  # this matches rating IDs, not pathnames
        print(rating_id)
    print(f"\nRating pathnames in file {dss.name}")
    for pathname in dss.catalog("RATING", pathnames=True):
        print(pathname)
    print(f"\nAll pathnames in file {dss.name}")
    for pathname in dss.catalog():
        print(pathname)

Rating IDs in DSS file C:\TEMP\rating_set.dss
KEYS.Elev;Stor.Linear.Production

Rating pathnames in file C:\TEMP\rating_set.dss
/SWT/KEYS/Elev;Stor/Rating-Body-2009-01-14T06:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Body-2011-10-19T05:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Body-2020-08-01T05:00:00Z/Linear/Production/

All pathnames in file C:\TEMP\rating_set.dss
/SWT//Elev;Stor/Rating-Template/Linear//
/SWT/KEYS/Elev;Stor/Rating-Specification/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Points-2009-01-14T06:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Body-2009-01-14T06:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Points-2011-10-19T05:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Body-2011-10-19T05:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Points-2020-08-01T05:00:00Z/Linear/Production/
/SWT/KEYS/Elev;Stor/Rating-Body-2020-08-01T05:00:00Z/Linear/Production/


### Use LocalRatingSet from HEC-DSS to Rate and ReverseRate TimeSeries

In [5]:
dss_file_name = (
    "C:/TEMP/rating_set.dss"
    if platform.system() == "Windows"
    else "/var/tmp/rating.dss"
)
with DssDataStore.open(dss_file_name, read_only=False) as dss:
    times = {"retrieve": {}, "rate": {}, "reverse_rate": {}, "total": {}}
    methods = ["LAZY", "EAGER"]
    for method in methods:
        # ------------------------------------------------ #
        # get a rating set from the dss file               #
        #                                                  #
        # method="REFERENCE" is not valid here             #
        #                                                  #
        # methods "LAZY" (default) and "EAGER" return a    #
        # LocalRatingSet that perform the ratings in local #
        # code. the difference is:                         #
        # - "LAZY" doesn't retrieve table data until it is #
        #   actually needed (prevents loading lots of      #
        #   data that is never used)                       #
        # - "EAGER"  retrieves all of the table data up    #
        #   front                                          #
        # ------------------------------------------------ #
        ts1 = datetime.now()
        rating_set = cast(
            LocalRatingSet,
            dss.retrieve(
                "KEYS.Elev;Stor.Linear.Production", office="SWT", method=method
            ),
        )
        ts2 = datetime.now()
        times["retrieve"][method] = str(ts2 - ts1)
        # ---------------------------------------------- #
        # rate the time series (retrieved 2 cells above) #
        # ---------------------------------------------- #
        ts3 = datetime.now()
        stor_ts = cast(TimeSeries, rating_set.rate(elev_ts))
        ts4 = datetime.now()
        times["rate"][method] = str(ts4 - ts3)
        print(f"\n{stor_ts}")
        print(stor_ts.data)
        # ---------------------------------- #
        # reverse rate the rated time series #
        # ---------------------------------- #
        ts5 = datetime.now()
        elev_ts2 = cast(TimeSeries, rating_set.reverse_rate(stor_ts))
        ts6 = datetime.now()
        times["reverse_rate"][method] = str(ts6 - ts5)
        times["total"][method] = str((ts2 - ts1) + (ts4 - ts3) + (ts6 - ts5))
        print(f"\n{elev_ts2}")
        print(elev_ts2.data)

    print("\n\t\tLAZY\t\tEAGER")
    print(f"retrieve\t{chr(9).join([times['retrieve'][m] for m in methods])}")
    print(f"rate\t\t{chr(9).join([times['rate'][m] for m in methods])}")
    print(f"reverse_rate\t{chr(9).join([times['reverse_rate'][m] for m in methods])}")
    print(f"\n\t\t{chr(9).join([times['total'][m] for m in methods])}")


KEYS.Stor.Inst.1Hour.0.Ccp-Rev 24 values in ac-ft
                                  value  quality
time                                            
2025-10-21 12:00:00-05:00  436568.72214        0
2025-10-21 13:00:00-05:00  436020.41292        0
2025-10-21 14:00:00-05:00  436203.18266        0
2025-10-21 15:00:00-05:00  435472.10370        0
2025-10-21 16:00:00-05:00  434741.02474        0
2025-10-21 17:00:00-05:00  433872.00388        0
2025-10-21 18:00:00-05:00  433528.87832        0
2025-10-21 19:00:00-05:00  432671.06442        0
2025-10-21 20:00:00-05:00  432156.37608        0
2025-10-21 21:00:00-05:00  432156.37608        0
2025-10-21 22:00:00-05:00  432327.93886        0
2025-10-21 23:00:00-05:00  432327.93886        0
2025-10-22 00:00:00-05:00  432842.62720        0
2025-10-22 01:00:00-05:00  432842.62720        0
2025-10-22 02:00:00-05:00  433185.75276        0
2025-10-22 03:00:00-05:00  433185.75276        0
2025-10-22 04:00:00-05:00  433528.87832        0
2025-10-22 05:00:0